# Learning from Point Clouds

In [ ]:
import copy
import numpy as np
from PIL import Image
import open3d as o3d
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from plotly.colors import sequential

In [ ]:
pcd = o3d.io.read_point_cloud("../data/bun_zipper_res2.ply")
pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(11))
pcd.orient_normals_consistent_tangent_plane(10)

mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=5, n_threads=16)
mesh.remove_duplicated_vertices()
mesh.remove_unreferenced_vertices()
mesh.remove_duplicated_triangles()
mesh.remove_degenerate_triangles()

vertices = np.asarray(mesh.vertices)
triangles = np.asarray(mesh.triangles)
x = vertices[:, 0]
y = vertices[:, 1]
z = vertices[:, 2]

centroids = np.random.choice(len(triangles), 250)
triangle_list = list()
for c in centroids:
    triangle_list.append(triangles[c-5:c+5, :])
sub_mesh = o3d.geometry.TriangleMesh()
sub_mesh.vertices = mesh.vertices
sub_mesh.vertex_normals = mesh.vertex_normals
sub_mesh.triangles = o3d.utility.Vector3iVector(np.concatenate(triangle_list))
sub_mesh.translate([0.2, 0, 0])

fig = go.Figure(ff.create_trisurf(x=x, y=y, z=z,
                                  simplices=triangles,
                                  plot_edges=False,
                                  edges_color="black",
                                  colormap=['#000004', '#180f3d', '#440f76', '#721f81', '#9e2f7f',
                                            '#cd4071', '#f1605d', '#fd9668', '#feca8d', '#fcfdbf'],
                                  show_colorbar=False).data)

camera = dict(eye=dict(x=-1e-7, y=-1e-5, z=2))
fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  height=700,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_camera=camera,
                  scene_dragmode="orbit",
                  showlegend=False)

In [ ]:
# Save figure
pio.write_html(fig,
               file=f"../_includes/figures/bunny_mesh.html",
               full_html=False,
               include_plotlyjs='cdn')

In [ ]:
pcd = mesh.sample_points_poisson_disk(number_of_points=2000, init_factor=5)
sub_pcd = sub_mesh.sample_points_poisson_disk(number_of_points=1000, init_factor=5)

points = np.asarray(pcd.points)
sub_points = np.asarray(sub_pcd.points)

fig = go.Figure([go.Scatter3d(x=points[:, 0],
                              y=points[:, 1],
                              z=points[:, 2],
                              mode="markers",
                              hovertemplate="<b>Point</b><br>x: %{x}<br>y: %{y}<br>z: %{z}<extra></extra>",
                              hoverlabel=dict(bgcolor="white"),
                              marker=dict(size=5,
                                          color=points[:, 2],
                                          colorscale='magma')),
                 go.Scatter3d(x=sub_points[:, 0],
                              y=sub_points[:, 1],
                              z=sub_points[:, 2],
                              mode="markers",
                              hovertemplate="<b>Point</b><br>x: %{x}<br>y: %{y}<br>z: %{z}<extra></extra>",
                              hoverlabel=dict(bgcolor="white"),
                              marker=dict(size=5,
                                          color=-sub_points[:, 2],
                                          colorscale='magma'))])

camera = dict(eye=dict(x=1, y=1, z=1.5),
              up=dict(x=0, y=1, z=0))
fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  height=500,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_camera=camera,
                  scene_dragmode="orbit",
                  showlegend=False)

In [ ]:
# Save figure
pio.write_html(fig,
               file=f"../_includes/figures/bunny_pcd.html",
               full_html=False,
               include_plotlyjs='cdn')

In [ ]:
def sphere(xyz, radius=1):
    mesh = o3d.geometry.TriangleMesh.create_sphere(radius=radius)
    mesh.translate([xyz[0], xyz[1], xyz[2]])

    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)
    x = vertices[:, 0]
    y = vertices[:, 1]
    z = vertices[:, 2]
    i = triangles[:, 0]
    j = triangles[:, 1]
    k = triangles[:, 2]
    
    return x, y, z, i, j, k


# Adapted from https://codereview.stackexchange.com/questions/179561/farthest-point-algorithm-in-python
def farthest_point_sampling(points, k):
    # Initialization
    farthest_points = np.zeros((k, 3))  # Pre-allocate array
    farthest_points[0] = points[np.random.randint(len(points))]  # Start with random point
    distances = ((farthest_points[0] - points) ** 2).sum(axis=1)  # Compute distance to all other points
    
    # Repeat for each subsequent point
    for i in range(1, k):
        farthest_points[i] = points[np.argmax(distances)]  # Find farthest point to current point
        distances = np.minimum(distances, ((farthest_points[i] - points) ** 2).sum(axis=1))
    return farthest_points


data = [go.Scatter3d(x=points[:, 0],
                     y=points[:, 1],
                     z=points[:, 2],
                     mode="markers",
                     hoverinfo="none",
                     marker=dict(size=5,
                                 color=points[:, 2],
                                 colorscale="magma"))]

colors = copy.copy(sequential.Blues)
for point in farthest_point_sampling(points, 9):
    x, y, z, i, j, k = sphere(point, radius=0.06)
    data.append(go.Mesh3d(x=x, y=y, z=z,
                          i=i, j=j, k=k,
                          hoverinfo="none",
                          color="blue",
                          opacity=0.05,
                          lighting=dict(fresnel=5, roughness=0, specular=2),
                          showscale=False))
    
x, y, z, i, j, k = sphere(np.mean(points, axis=0), radius=0.12)
data.append(go.Mesh3d(x=x, y=y, z=z,
                      i=i, j=j, k=k,
                      hoverinfo="none",
                      color="blue",
                      opacity=0.1,
                      lighting=dict(fresnel=5, roughness=0, specular=2),
                      showscale=False))

fig = go.Figure(data)
camera = dict(eye=dict(x=0.7, y=0.7, z=1.2),
              up=dict(x=0, y=1, z=0))
fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  height=700,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_camera=camera,
                  scene_dragmode="orbit",
                  showlegend=False)

In [ ]:
# Save figure
pio.write_html(fig,
               file=f"../_includes/figures/bunny_with_spheres.html",
               full_html=False,
               include_plotlyjs='cdn')

In [ ]:
points = np.random.randn(100, 3)
points /= np.linalg.norm(points, axis=0)

data = [go.Scatter3d(x=points[:, 0],
                     y=points[:, 1],
                     z=points[:, 2],
                     mode="markers",
                     hoverinfo="none",
                     marker=dict(size=5,
                                 color=-points[:, 2],
                                 colorscale="magma"))]

for point in farthest_point_sampling(points, 3):
    x, y, z, i, j, k = sphere(point * 0.5, radius=0.2)
    data.append(go.Mesh3d(x=x, y=y, z=z,
                          i=i, j=j, k=k,
                          hoverinfo="none",
                          color="red",
                          opacity=0.1,
                          lighting=dict(fresnel=5, roughness=0, specular=2),
                          showscale=False))
    
x, y, z, i, j, k = sphere(np.mean(points, axis=0), radius=0.4)
data.append(go.Mesh3d(x=x, y=y, z=z,
                      i=i, j=j, k=k,
                      hoverinfo="none",
                      color="blue",
                      opacity=0.1,
                      lighting=dict(fresnel=5, roughness=0, specular=2),
                      showscale=False))
    
fig = make_subplots(rows=1,
                    cols=2,
                    column_widths=[0.5, 0.5],
                    horizontal_spacing=0.05,
                    vertical_spacing=0,
                    specs=[[dict(type="Mesh3d"), dict(type="Mesh3d")]])

for d in data:
    fig.add_trace(d, row=1, col=1)

fig.add_trace(data[0], row=1, col=2)
for r, c in zip([0.1, 0.2, 0.4], ["green", "red", "blue"]):
    x, y, z, i, j, k = sphere(np.mean(points, axis=0), radius=r)
    fig.add_trace(go.Mesh3d(x=x, y=y, z=z,
                           i=i, j=j, k=k,
                           hoverinfo="none",
                           color=c,
                           opacity=0.1,
                           lighting=dict(fresnel=5, roughness=0, specular=2),
                           showscale=False), row=1, col=2)
    
fig.update_layout(scene1=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  scene2=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  height=500,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_dragmode="orbit",
                  showlegend=False)

In [ ]:
# Save figure
pio.write_html(fig,
               file=f"../_includes/figures/mrg_vs_msg.html",
               full_html=False,
               include_plotlyjs='cdn')